# Rain Hyperparameter Initialization

The purpose of this notebook is to examine the hyperparameter associated with rain in the FMDA RNN model. The current architecture is to have an additive effect of rain on fuel moisture in the initiated model. There is a hyperparameter associated with this effect that we wish to set from a physics-based model.

## Background: Physics Model from Vanderkamp 2014

Vanderkamp (2014) establish the following physics-based model of the affect of rain on fuels. The depth of a stick is modeled, and there is assumed to be radial symmetry and symmetry along the length of the stick. During rain, the outer layer of the stick absorbs the rain. The amount of absorbed rain $P_{abs}$, in units of $kg/s$, is calculated from the diamter of the stick $2r$, length $l$ and the incident precipitation rate $P_{inc}$ (in units of $kg s^{-1}m^{-2}$):

$P_{abs}=2r l P_{inc}$

Additionally, the moisture is limited to a max value $m_{max}$, which is "determined through model calibration". For 10-h fuels, they arrived on $m_{max}=76.8\%$. This value was valibrated at a single study location.

## Translate Vanderkamp Model to our context

### Calculate Expected Change from 1mm Rain

In order to apply this physical principle to the physics-initiated RNN, we use this formula to calculate the expected increase in fuel moisture content from 1mm of rain over an hour. Additionally, we are working with 10h fuels, which correspond to roughly 1/2 inch diamter sticks, or $0.0127 m$

1mm of rain spread over $1m^2$ over an hour corresponds to 1kg of water per hour, so let $P_{inc}=1kg hr^{-1}=1/3600 \; kg s^{-1}$.
* 1mm is .001m, and if spread over $1m^2$, we have a 3-d volume of water .001 $m^3$
* The density of water is approximately 1,000kg per $m^3$, so $1,000 kgm^{-3}\cdot .001 m^3=1kg$

The percent change in fuel moisture content per hour, in units of $hr{-1}$ is calculated assuming an initial dry weight of $M$ kg. Denote this term $K$:

$$
K = \frac{\text{Weight of water per hour}\;kg\;hr^{-1}}{M\; kg},\quad\text{Units: }hr^{-1}
$$
$$
= \frac{2lr P_{inc}\; kg\;s^{-1} \cdot 3600 s\;hr{-1}}{M \; kg}
$$

The mass of the stick is $M=\text{Volume}\cdot \text{Density}$. The volume can be calculated geometrically, and the Density comes from literature. [Research suggests](https://www.engineeringtoolbox.com/wood-density-d_40.html) that the density of woods varies from 300 to 900 $kg\;m{-3}$. 

$$
K = \frac{2lr}{\pi r^2 l \rho}
$$
$$
= \frac{2}{\pi r \rho}=\frac{4}{\pi d \rho}
$$

Here, $d$ is the diameter of the stick, $0.0127 m$. So substituting this into the equation, with wood densities of 300 to 900 $kg \; m^{-3}$, we get:

$$
K = 0.11 \text{ to } 0.33
$$

In percentage terms this is $11$ to $33$ percent fuel moisture per mm of rain in an hour. As a feasibility check, working with the denser wood estimate a stick would have moisture content of 100 if exposed to 9mm of rain in an hour. [AMS](https://glossary.ametsoc.org/wiki/Rain) classifies this as very heavy rain, so it is feasible that a dry stick exposed to very heavy rain for an hour would absorb up to its weight in water. Furthermore, the stick would reach saturation level of 250% from about 23 mm of rain per hour, which corresponds to a torrential downpour. This again is feasible.

For RAWS 10-hour sensors specifically, they utilize a wooden dowel made of Ponderosa pine with a weight of 45g (or .045 kg), according to the 2017 brochure from Campbell Scientific. The dimensions are .5in by 20in, or .0127m by .508m. Assuming this is the pure dry weight, the density calculation (in $kgm^{-3}$) would be:

$$
\begin{align}
    \text{Density} &= \frac{\text{Mass}}{\text{Volume}}\\
    & = \frac{0.045 kg}{\pi\cdot (.0127/2)^2\cdot.508 m^3}\\
    & = 700 kgm^{-3}
\end{align}
$$

This would correspond to $K=0.143$

### Implementing in the RNN Model

In the RNN model, the equilibrium inputs are scaled by a parameter $S$. Currently (1/5/2024) this is calculated from the maximum observed fm in the input data:

$$
\begin{align}
E_d &= E_d/S\\
E_w &= E_w/S\\
fm &= fm/S
\end{align}
$$

This results in fuel moisture values that are roughly scaled to be decimal values instead of in percentage terms (i.e. 0.1 instead of 10).

Currently (Jan 5 2024), the initialized model adds a constant effect to fuel moisture per mm of rain. So the fuel moisture at time $t+1$, $m_{t+1}$, is calculated using the ODE time lag model on the moisture at time $t$, denote this $f(m_t, E_t)$, plus the additive effect of rain. Let the fm increase per mm of rain parameter be $K$ and the rain at time $t$ is $R_t$:

$$
m_{t+1}=f(m_t, E_t)+K\cdot R_t
$$

The additive rain term needs to be scaled properly. To figure out the scaling for the rain term, we walk through an example calculation below.

Suppose the initial moisture is zero, or $m_0=0$. Note this is how the RNN initializes the hidden state by default. Also suppose the equilibrium moisture at $t=0$ is $10$ ($E_d$ and $E_w$ are averaged on input, so ignore this for now) and the scale parameter is $30$, which is a feasible maximum observable moisture value. So the scaled input at $t=0$ is $.3$. The ODE portion of the model is:

$$
f(m_t, E_t) = e^{-.1}\cdot m_t + (1-e^{-.1})\cdot E_t
$$
$$
= (1-e^{-.1})\cdot 0.3 \approx 0.03
$$

So, with zero rain the fuel moisture $m_{t+1}=0.03$, or 3%.

If we assume $1mm$ of rain at $t=0$, and a middle-valued wood density of $600 \;kg\;m{-3}$, we would have:

$$
K\cdot R_0=.167 \cdot 1 = 0.167
$$

So, the modeled fuel moisture at time $t=1$ would be:

$$
m_{t+1}=0.03 + 0.167 = 0.17
$$

So in this formulation, the rain would be unscaled and the percent change in fuel moisture per mm of rain would be in decimal terms, or about $0.167$.

## Setup

In [ ]:
import sys
sys.path.append("..")

In [ ]:
# Setup
import reproducibility

# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import tensorflow as tf
import pickle, os

from data_funcs import load_and_fix_data, plot_data
from moisture_rnn import create_rnn_data, train_rnn, rnn_predict

In [ ]:
# Data
# Change directory for data read/write

dict_file='../data/raws_CO_202306.pickle' # input path of FMDA dictionaries
reproducibility_file='../data/reproducibility_dict.pickle'

# read test datasets
test_dict={}
test_dict.update(load_and_fix_data(dict_file))
print(test_dict.keys())

repro_dict={}
repro_dict.update(load_and_fix_data(reproducibility_file))
print(repro_dict.keys())
# Build Case Data
id = "CPTC2_202306010000"
case_data=test_dict[id]
case_data["hours"]=len(case_data['fm'])
case_data["h2"]=int(24*20)

## References

* Vanderkamp 2014
* Mandel 2014
* USDA Stick Diam: https://www.fs.usda.gov/t-d/pubs/htmlpubs/htm05512347/
* AMS Rain: https://glossary.ametsoc.org/wiki/Rain
* RAWS Sensor specifications: https://s.campbellsci.com/documents/us/product-brochures/b_cs506_cs205.pdf
    * Stick Specs: https://www.campbellsci.com/pn26601